In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of document_model failed: Traceback (most recent call last):
  File "/Users/harryzhao/Library/Caches/pypoetry/virtualenvs/opensearch-poc-nuZIkl0S-py3.12/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/harryzhao/Library/Caches/pypoetry/virtualenvs/opensearch-poc-nuZIkl0S-py3.12/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/importlib/__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 995, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "/Users/harryzhao/Documents/opensearch_poc/opensearch_poc/document_

In [2]:
from opensearchpy import OpenSearch


In [21]:

from opensearchpy.helpers.document import Document, InnerDoc
from opensearchpy.helpers.field import Text, Double, Nested, Date, DenseVector
class ContentChunk(InnerDoc):
    chunk_content= Text()
    chunk_vector=DenseVector(dims=3)

class RetrivalDocument(Document):
    title= Text()
    content=Text()
    content_vector = DenseVector(dims=3)
    source_type = Text()
    time=Date()
    content_chunks = Nested(ContentChunk)


In [12]:
host = 'localhost'
port = 9200
auth = ('admin', 'Danswer123?') # For testing only. Don't store credentials in code.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)


In [26]:
# index_name = 'document-index'
# index_body = {
#   'settings': {
#     'index': {
#       'number_of_shards': 1
#     }
#   }
# }

# response = client.indices.create(index_name, body=index_body)


In [27]:
# RetrivalDocument.init(using=client, index='document-index') #Couldnt get this to work for now, skipping

In [28]:
# from document_model import RetrivalDocument, ContentChunk


# sample_doc_1 = RetrivalDocument(title="Sample Title", content="Sample Content", content_vector=[1,2,3], source_type="Slack", time="2020-01-01", content_chunks=[ContentChunk(chunk_content="Sample", chunk_vector=[0,1,2]), ContentChunk(" Content", chunk_vector=[3,4,5])])

In [244]:
index_name = 'actual-index9'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 1,
      "knn": True #need this for knn to work
    }
  },
  "mappings": {
    "properties": {
      "title":    { "type" : "text" },
      "content":     { "type" : "text" },
      "content_vector":{ "type" : "knn_vector", "dimension": 3,"method": { #need all these for knn to work
          "name": "hnsw",
          "space_type": "l2",
          "engine": "lucene",
          "parameters": {
            "ef_construction": 128,
            "m": 24
          }
        } },
      "source_type":{ "type" : "text" },
      "time": {"type": "unsigned_long", }, #their dates are stored with some custom class 
      #https://javadoc.io/doc/org.elasticsearch/elasticsearch/7.5.0/org/elasticsearch/script/JodaCompatibleZonedDateTime.html
      # I just did unsigned_long for now to unblock
      "content_chunks": {
          "type" : "nested",
          "properties":{
              "content_chunk_text": {"type": "text"},
              "content_chunk_vector" : {"type": "knn_vector", "dimension":3, "method": {
          "name": "hnsw",
          "space_type": "l2",
          "engine": "lucene",
          "parameters": {
            "ef_construction": 128,
            "m": 24
          }
        }}
          }
      }
    }
  }
}



In [227]:
from datetime import datetime
from time import strftime

def convert_to_epoch_time(time_str: str):
    p = '%Y-%m-%dT%H:%M:%S.%fZ'
    
    epoch = datetime(1970, 1, 1)
    return (datetime.strptime(time_str, p) - epoch).total_seconds()

In [228]:
convert_to_epoch_time("2024-06-16T11:54:04.000Z")

1718538844.0

In [245]:
response = client.indices.create(index_name, body=index_body)


In [246]:
body = {
    "title": "to be or not to be",
    "content": """To be, or not to be, that is the question: 
    Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles
And by opposing end them. """,
    "content_vector": [1,2,3],
    "source_type": "slack",
    "time": convert_to_epoch_time("2024-06-16T11:54:04.000Z"),
    "content_chunks":[
        {
            "content_chunk_text": "To be, or not to be, that is the question",
            "content_chunk_vector": [0,1,2]
        },
        {
            "content_chunk_text": """Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles
And by opposing end them.""",
            "content_chunk_vector": [4,5,6]
        }
    ]
}


client.index(index=index_name, body=body)

{'_index': 'actual-index9',
 '_id': '3WxeIpAB0X4UA6bwhBzk',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 2, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [247]:
body = {
    "title": "All the world’s a stage",
    "content": """All the world’s a stage,
And all the men and women merely players;
They have their exits and their entrances;
And one man in his time plays many parts,
His acts being seven ages. """,
    "content_vector": [101,102,103],
    "source_type": "confluence",
    "time": convert_to_epoch_time("2024-01-01T11:54:04.000Z"),
    "content_chunks":[
        {
            "content_chunk_text": """All the world’s a stage,
And all the men and women merely players;""",
            "content_chunk_vector": [100,101,102]
        },
        {
            "content_chunk_text": """They have their exits and their entrances;
And one man in his time plays many parts,
His acts being seven ages. """,
            "content_chunk_vector": [104,105,106]
        }
    ]
}
client.index(index=index_name, body=body)

{'_index': 'actual-index9',
 '_id': '3mxeIpAB0X4UA6bwoxz2',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 2, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

In [248]:
body = {
    "title": "O Ceremony",
    "content": """O ceremony, show me but thy worth!
What is thy soul of adoration?
Art thou aught else but place, degree and form,
Creating awe and fear in other men?
Wherein thou art less happy being fear’d
Than they in fearing.
What drink’st thou oft, instead of homage sweet,
But poison’d flattery?  """,
    "content_vector": None,
    "source_type": "slack",
    "time": convert_to_epoch_time("2020-01-01T11:54:04.000Z"),
    "content_chunks":[
        {
            "content_chunk_text": """O ceremony, show me but thy worth!
What is thy soul of adoration?
Art thou aught else but place, degree and form,
Creating awe and fear in other men?""",
            "content_chunk_vector": [200,201,202]
        },
        {
            "content_chunk_text": """Wherein thou art less happy being fear
Than they in fearing.
What drink’st thou oft, instead of homage sweet,
But poison’d flattery?""",
            "content_chunk_vector": [204,205,206]
        }
    ]
}
client.index(index=index_name, body=body)

{'_index': 'actual-index9',
 '_id': '32xeIpAB0X4UA6bwrBxP',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 2, 'failed': 0},
 '_seq_no': 2,
 '_primary_term': 1}

In [411]:
#keyword search on content and content_chunk
search_body = {
  "query": {
    "bool": {
      "should": [
        {
          "function_score": {
            "query": {
              "match_phrase": {
                "content": "fear"
              }
            },
            "boost": 1.0
          }
        },
        {
          "function_score": {
            "query": {
              "nested": {
                "path": "content_chunks",
                "query": {
                  "match_phrase": {
                    "content_chunks.content_chunk_text": "fear"
                  }
                },
                "inner_hits": {
                  "name": "matching_chunks",
                  "size": 10,
                  "sort": [
                    {
                      "_score": {
                        "order": "desc"
                      }
                    }
                  ],
                  "highlight": {
                    "fields": {
                      "content_chunks.content_chunk_text": {}
                    }
                  }
                }
              }
            },
            "boost": 0.5 #Notice you can affect how sub-queries weigh
          }
        }
      ]
    }
  }
}

In [412]:



client.search(body=search_body, index=index_name)

{'took': 12,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.8468283,
  'hits': [{'_index': 'actual-index9',
    '_id': '32xeIpAB0X4UA6bwrBxP',
    '_score': 1.8468283,
    '_source': {'title': 'O Ceremony',
     'content': 'O ceremony, show me but thy worth!\nWhat is thy soul of adoration?\nArt thou aught else but place, degree and form,\nCreating awe and fear in other men?\nWherein thou art less happy being fear’d\nThan they in fearing.\nWhat drink’st thou oft, instead of homage sweet,\nBut poison’d flattery?  ',
     'content_vector': None,
     'source_type': 'slack',
     'time': 1577879644.0,
     'content_chunks': [{'content_chunk_text': 'O ceremony, show me but thy worth!\nWhat is thy soul of adoration?\nArt thou aught else but place, degree and form,\nCreating awe and fear in other men?',
       'content_chunk_vector': [200, 201, 202]},
      {'content_chunk_text': 'W

In [324]:
#knn search on both the content_vector and content_chunks,content_chunk_vector
search_body= {
  "query": {
    "bool": {
      "should": [
        {
          "knn": {
            "content_vector": {
              "vector": [0,1,2],
              "k": 2
            }
          }
        },
        {
          "nested": {
            "path": "content_chunks",
            "query": {
              "knn": {
                "content_chunks.content_chunk_vector": {
                  "vector": [0,1,2],
                  "k": 2
                }
              }
            },
            "inner_hits": {
              "name": "content_chunk_hits",
              "size": 10  
            }
          }
        }
      ]
    }
  }
}


In [325]:
#notice the score is an sum of outer main content score and max of inner hits scores.
#switch vector to [1,2,3] and notice the score changes.
client.search(body=search_body, index=index_name)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.25,
  'hits': [{'_index': 'actual-index9',
    '_id': '3WxeIpAB0X4UA6bwhBzk',
    '_score': 1.25,
    '_source': {'title': 'to be or not to be',
     'content': "To be, or not to be, that is the question: \n    Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune,\nOr to take arms against a sea of troubles\nAnd by opposing end them. ",
     'content_vector': [1, 2, 3],
     'source_type': 'slack',
     'time': 1718538844.0,
     'content_chunks': [{'content_chunk_text': 'To be, or not to be, that is the question',
       'content_chunk_vector': [0, 1, 2]},
      {'content_chunk_text': "Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune,\nOr to take arms against a sea of troubles\nAnd by opposing end them.",
       'content_chunk_vector': [4, 5, 6]}]},

In [267]:
#A vector that's close to two different documents (see the search below for date boosting difference)
search_body= {
  "query": {
    "bool": {
      "should": [
        {
          "knn": {
            "content_vector": {
              "vector": [55,55,55],
              "k": 2
            }
          }
        },
        {
          "nested": {
            "path": "content_chunks",
            "query": {
              "knn": {
                "content_chunks.content_chunk_vector": {
                  "vector": [55,55,55],
                  "k": 2
                }
              }
            },
            "inner_hits": {
              "name": "content_chunk_hits",
              "size": 10  
            }
          }
        }
      ]
    }
  }
}

In [268]:
client.search(body=search_body, index=index_name)

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 0.0003082851,
  'hits': [{'_index': 'actual-index9',
    '_id': '3mxeIpAB0X4UA6bwoxz2',
    '_score': 0.0003082851,
    '_source': {'title': 'All the world’s a stage',
     'content': 'All the world’s a stage,\nAnd all the men and women merely players;\nThey have their exits and their entrances;\nAnd one man in his time plays many parts,\nHis acts being seven ages. ',
     'content_vector': [101, 102, 103],
     'source_type': 'confluence',
     'time': 1704110044.0,
     'content_chunks': [{'content_chunk_text': 'All the world’s a stage,\nAnd all the men and women merely players;',
       'content_chunk_vector': [100, 101, 102]},
      {'content_chunk_text': 'They have their exits and their entrances;\nAnd one man in his time plays many parts,\nHis acts being seven ages. ',
       'content_chunk_vector': [104, 105, 106

In [356]:
#date boosting 
# (I did crank up the decay factor to show the difference, as the difference in days between the two documents are not very significant)
search_body= {
  "query": {
    "function_score": {
      "query": {
        "bool": {
          "should": [
            {
              "knn": {
                "content_vector": {
                  "vector": [55,55,55],
                  "k": 2
                }
              }
            },
            {
              "nested": {
                "path": "content_chunks",
                "query": {
                  "knn": {
                    "content_chunks.content_chunk_vector": {
                      "vector": [55,55,55],
                      "k": 2
                    }
                  }
                },
                "inner_hits": {
                  "name": "content_chunk_hits",
                  "size": 10
                }
              }
            }
          ]
        }
      },
      "functions": [
        {
            
          "script_score": {
            "script": {
                #this returns a coefficient
              "source": """
                double decayFactor = params.decayFactor;
                long documentAgeMillis = params.now - doc['time'].value;
                double documentAgeDays = documentAgeMillis / 86400000.0; 
                double decay = Math.max(1 / (1 + decayFactor * documentAgeDays), 0.75);
                return  decay;
              """,
              "params": {
                "decayFactor": 2, 
                "now": 1718538844
              }
            }
          }
        }
      ],
      #this multiplies the coefficient returned (per document) on its score.
      "boost_mode": "multiply"
    }
  }
}

In [357]:
client.search(body=search_body, index=index_name)

{'took': 8,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 0.00025190398,
  'hits': [{'_index': 'actual-index9',
    '_id': '3WxeIpAB0X4UA6bwhBzk',
    '_score': 0.00025190398,
    '_source': {'title': 'to be or not to be',
     'content': "To be, or not to be, that is the question: \n    Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune,\nOr to take arms against a sea of troubles\nAnd by opposing end them. ",
     'content_vector': [1, 2, 3],
     'source_type': 'slack',
     'time': 1718538844.0,
     'content_chunks': [{'content_chunk_text': 'To be, or not to be, that is the question',
       'content_chunk_vector': [0, 1, 2]},
      {'content_chunk_text': "Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune,\nOr to take arms against a sea of troubles\nAnd by opposing end them.",
       'content_chunk_vect

In [344]:
#filter against source
search_body = {
  "query": {
    "function_score": {
      "query": {
        "bool": {
          "should": [
            {
              "knn": {
                "content_vector": {
                  "vector": [55, 55, 55],
                  "k": 2
                }
              }
            },
            {
              "nested": {
                "path": "content_chunks",
                "query": {
                  "knn": {
                    "content_chunks.content_chunk_vector": {
                      "vector": [55, 55, 55],
                      "k": 2
                    }
                  }
                },
                "inner_hits": {
                  "name": "content_chunk_hits",
                  "size": 10
                }
              }
            }
          ],
          #filter for source
          "filter": {
            "term": {
              "source_type": "slack"
            }
          }
        }
      },
      "functions": [
        {
          "script_score": {
            "script": {
              "source": """
                double decayFactor = params.decayFactor;
                long documentAgeMillis = params.now - doc['time'].value;
                double documentAgeDays = documentAgeMillis / 86400000.0; 
                double decay = Math.max(1 / (1 + decayFactor * documentAgeDays), 0.75);
                double originalScore = _score;
                return decay ;
              """,
              "params": {
                "decayFactor": 0.0001, 
                "now": 1718563151798 
              }
            }
          }
        }
      ],
      "boost_mode": "multiply"
    }
  }
}

In [345]:
client.search(body=search_body, index=index_name)

{'took': 14,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 0.00018892798,
  'hits': [{'_index': 'actual-index9',
    '_id': '3WxeIpAB0X4UA6bwhBzk',
    '_score': 0.00018892798,
    '_source': {'title': 'to be or not to be',
     'content': "To be, or not to be, that is the question: \n    Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune,\nOr to take arms against a sea of troubles\nAnd by opposing end them. ",
     'content_vector': [1, 2, 3],
     'source_type': 'slack',
     'time': 1718538844.0,
     'content_chunks': [{'content_chunk_text': 'To be, or not to be, that is the question',
       'content_chunk_vector': [0, 1, 2]},
      {'content_chunk_text': "Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune,\nOr to take arms against a sea of troubles\nAnd by opposing end them.",
       'content_chunk_vec

In [405]:
#Hybrid search naive
search_body={
  "query": {
    "bool": {
      "should": [
        {
          "function_score": {
            "query": {
              "bool": {
                "should": [
                  {
                    "match_phrase": {
                      "content": "fear"
                    }
                  },
                  {
                    "nested": {
                      "path": "content_chunks",
                      "query": {
                        "match_phrase": {
                          "content_chunks.content_chunk_text": "fear"
                        }
                      },
                      "inner_hits": {
                        "name": "matching_chunks",
                        "size": 10,
                        "sort": [
                          {
                            "_score": {
                              "order": "desc"
                            }
                          }
                        ],
                        "highlight": {
                          "fields": {
                            "content_chunks.content_chunk_text": {}
                          }
                        }
                      }
                    }
                  }
                ]
              }
            },
            "boost": 0.3
          }
        },
        {
          "function_score": {
            "query": {
              "bool": {
                "should": [
                  {
                    "knn": {
                      "content_vector": {
                        "vector": [0,1,2],
                        "k": 2
                      }
                    }
                  },
                  {
                    "nested": {
                      "path": "content_chunks",
                      "query": {
                        "knn": {
                          "content_chunks.content_chunk_vector": {
                            "vector": [0,1,2],
                            "k": 2
                          }
                        }
                      },
                      "inner_hits": {
                        "name": "content_chunk_hits",
                        "size": 10
                      }
                    }
                  }
                ]
              }
            },
            "boost": 0.7
          }
        }
      ]
    }
  }
}

In [406]:
client.search(body=search_body, index=index_name)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 0.875,
  'hits': [{'_index': 'actual-index9',
    '_id': '3WxeIpAB0X4UA6bwhBzk',
    '_score': 0.875,
    '_source': {'title': 'to be or not to be',
     'content': "To be, or not to be, that is the question: \n    Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune,\nOr to take arms against a sea of troubles\nAnd by opposing end them. ",
     'content_vector': [1, 2, 3],
     'source_type': 'slack',
     'time': 1718538844.0,
     'content_chunks': [{'content_chunk_text': 'To be, or not to be, that is the question',
       'content_chunk_vector': [0, 1, 2]},
      {'content_chunk_text': "Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune,\nOr to take arms against a sea of troubles\nAnd by opposing end them.",
       'content_chunk_vector': [4, 5, 6]}]

In [362]:
pipeline_body = {
  "description": "Post processor for hybrid search",
  "phase_results_processors": [
    {
      "normalization-processor": {
        "normalization": {
          "technique": "min_max"
        },
        "combination": {
          "technique": "arithmetic_mean",
          "parameters": {
            "weights": [
              0.7,
              0.3
            ]
          }
        }
      }
    }
  ]
}
client.transport.perform_request(
    "PUT",
    "/_search/pipeline/norm_pipeline",
    body=pipeline_body
)

{'acknowledged': True}

In [360]:
search_body = {
  "query": {
    "hybrid": {
      "queries": [
        {
    "bool": {
      "should": [
        {
          "knn": {
            "content_vector": {
              "vector": [0,1,2],
              "k": 2
            }
          }
        },
        {
          "nested": {
            "path": "content_chunks",
            "query": {
              "knn": {
                "content_chunks.content_chunk_vector": {
                  "vector": [0,1,2],
                  "k": 2
                }
              }
            },
            "inner_hits": {
              "name": "content_chunk_hits",
              "size": 10  
            }
          }
        }
      ]
    }
  },
              {
                "match": {
                  "content": "ceremony"
                }
              }
      ]
    }
  }
}

In [361]:
client.search(body=search_body, index=index_name, params={'search_pipeline': "norm_pipeline"})



{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 0.7,
  'hits': [{'_index': 'actual-index9',
    '_id': '32xeIpAB0X4UA6bwrBxP',
    '_score': 0.7,
    '_source': {'title': 'O Ceremony',
     'content': 'O ceremony, show me but thy worth!\nWhat is thy soul of adoration?\nArt thou aught else but place, degree and form,\nCreating awe and fear in other men?\nWherein thou art less happy being fear’d\nThan they in fearing.\nWhat drink’st thou oft, instead of homage sweet,\nBut poison’d flattery?  ',
     'content_vector': None,
     'source_type': 'slack',
     'time': 1577879644.0,
     'content_chunks': [{'content_chunk_text': 'O ceremony, show me but thy worth!\nWhat is thy soul of adoration?\nArt thou aught else but place, degree and form,\nCreating awe and fear in other men?',
       'content_chunk_vector': [200, 201, 202]},
      {'content_chunk_text': 'Wherein thou a

In [374]:
#notice that the scores are normalized against the entries
#change vector to [101,102,103] and notice the score does not change for vector
#Unfortunately this does not give the inner scores (on the chunks)
# all the scores (and normalization) is done on document level (which ig makes sense)
#because the way the queries are structured, the keyword and vector queries are two different objects
#therefore the score combinations/normalizations dont make sense unless they are at the document level
#ofc we can do post-processing on our backend and just do two queries, one for vector and one for keyword
search_body = {
  "query": {
    "hybrid": {
      "queries": [
        {
          "nested": {
            "path": "content_chunks",
            "query": {
              "knn": {
                "content_chunks.content_chunk_vector": {
                  "vector": [55,55,55],
                  "k": 2
                }
              }
            },
            "inner_hits": {
              "name": "content_chunk_hits",
              "size": 10  
            }
          }
        },
              {"nested": {
      "path": "content_chunks",
      "query": {
        "match_phrase": {
          "content_chunks.content_chunk_text": "ceremony"
        }
      },
      "inner_hits": {
        "name": "matching_chunks",
        "size": 10,
        "sort": [
          {
            "_score": {
              "order": "desc"
            }
          }
        ],
        "highlight": {
          "fields": {
            "content_chunks.content_chunk_text": {}
          }
        }
      }
    }}
      ]
    }
  }
}

In [375]:
client.search(body=search_body, index=index_name, params={'search_pipeline': "norm_pipeline"})


{'took': 7,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 0.7,
  'hits': [{'_index': 'actual-index9',
    '_id': '3mxeIpAB0X4UA6bwoxz2',
    '_score': 0.7,
    '_source': {'title': 'All the world’s a stage',
     'content': 'All the world’s a stage,\nAnd all the men and women merely players;\nThey have their exits and their entrances;\nAnd one man in his time plays many parts,\nHis acts being seven ages. ',
     'content_vector': [101, 102, 103],
     'source_type': 'confluence',
     'time': 1704110044.0,
     'content_chunks': [{'content_chunk_text': 'All the world’s a stage,\nAnd all the men and women merely players;',
       'content_chunk_vector': [100, 101, 102]},
      {'content_chunk_text': 'They have their exits and their entrances;\nAnd one man in his time plays many parts,\nHis acts being seven ages. ',
       'content_chunk_vector': [104, 105, 106]}]}},
   {'_index